In [2]:
import os
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, load_model

In [4]:
(x_train_full, y_train_full), (x_test, y_test) = cifar10.load_data()
x_train, x_valid = x_train_full[:-5000]/255, x_train_full[-5000:]/255
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]
print ("X_train shape: " + str(x_train.shape)  + 
    "\nY_train shape: " + str(y_train.shape) + 
    "\nX_test shape: " + str(x_test.shape)  + 
    "\nY_test shape: " + str(y_test.shape)  +
    "\nX_valid shape: " + str(x_valid.shape) + 
    "\nY_valid shape: " + str(y_valid.shape))

X_train shape: (45000, 32, 32, 3)
Y_train shape: (45000, 1)
X_test shape: (10000, 32, 32, 3)
Y_test shape: (10000, 1)
X_valid shape: (5000, 32, 32, 3)
Y_valid shape: (5000, 1)


In [100]:
def set_seed():
  InitSeed = 767
  tf.random.set_seed(InitSeed)
  np.random.seed(InitSeed)


def classifier_(input_):
  initializer = tf.keras.initializers.GlorotNormal(seed=767)
  output_ = keras.layers.GlobalAveragePooling2D()(input_)
  output_ = keras.layers.Flatten()(output_)
  output_ = keras.layers.BatchNormalization()(output_)
  output_prime = output_

  output_ = keras.layers.Dense(2048, kernel_initializer=initializer)(output_)
  output_ = keras.layers.BatchNormalization()(output_)
  output_ = keras.layers.Activation('relu')(output_)
  output_ = keras.layers.Dropout(0.3)(output_)

  output_ = keras.layers.Dense(2048, kernel_initializer=initializer)(output_)
  output_ = keras.layers.BatchNormalization()(output_)
  output_ = keras.layers.Activation('relu')(output_)
  output_ = keras.layers.Dropout(0.3)(output_)
  
  output_ = keras.layers.Dense(1024, kernel_initializer=initializer)(output_)
  output_ = keras.layers.BatchNormalization()(output_)
  output_ = keras.layers.Activation('relu')(output_)
  output_ = keras.layers.Dropout(0.3)(output_)

  output_ = keras.layers.Concatenate()([output_, output_prime])

  output_ = keras.layers.Dense(512, activation='relu', kernel_initializer=initializer)(output_)
  output_ = keras.layers.Dense(10, activation='softmax')((output_))
  return output_


def model_flow(input_):
  output_ = keras.layers.UpSampling2D(size=(7,7))(input_)
  output_ = tf.keras.applications.resnet.ResNet50(input_shape=(224, 224, 3),
                           include_top=False,
                           weights='imagenet')(output_)
  
  # output_ a fc1000 classification at this point
  # Add the self-designed classifier to produce 10-classes result
  output_ = classifier_(output_)
  return output_


def input_output_compile():
  # Shape of cifar10
  input_ = keras.layers.Input(shape=(32, 32, 3))
  output_ = model_flow(input_)
  clf = keras.Model(inputs=input_, outputs=output_)
  
  clf.compile(optimizer='SGD', loss='sparse_categorical_crossentropy',
              metrics = ['accuracy'])
  return clf

In [101]:
set_seed()
model = input_output_compile()

In [102]:
model.summary()

Model: "model_24"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_57 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 up_sampling2d_27 (UpSampling2D  (None, 224, 224, 3)  0          ['input_57[0][0]']               
 )                                                                                                
                                                                                                  
 resnet50 (Functional)          (None, 7, 7, 2048)   23587712    ['up_sampling2d_27[0][0]']       
                                                                                                  
 global_average_pooling2d_27 (G  (None, 2048)        0           ['resnet50[0][0]']        

In [103]:
set_seed()
hist = model.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=15, batch_size=64)

Epoch 1/15
704/704 [==============================] - 473s 665ms/step - loss: 0.4648 - accuracy: 0.8394 - val_loss: 2.3170 - val_accuracy: 0.3410
Epoch 2/15
704/704 [==============================] - 467s 663ms/step - loss: 0.1361 - accuracy: 0.9540 - val_loss: 0.2010 - val_accuracy: 0.9276
Epoch 3/15
704/704 [==============================] - 466s 663ms/step - loss: 0.0488 - accuracy: 0.9852 - val_loss: 0.1859 - val_accuracy: 0.9432
Epoch 4/15
704/704 [==============================] - 466s 663ms/step - loss: 0.0194 - accuracy: 0.9949 - val_loss: 0.2054 - val_accuracy: 0.9400
Epoch 5/15
704/704 [==============================] - 466s 662ms/step - loss: 0.0109 - accuracy: 0.9976 - val_loss: 0.2583 - val_accuracy: 0.9342
Epoch 6/15
704/704 [==============================] - 466s 662ms/step - loss: 0.0081 - accuracy: 0.9982 - val_loss: 0.9322 - val_accuracy: 0.8482
Epoch 7/15
704/704 [==============================] - 466s 663ms/step - loss: 0.0107 - accuracy: 0.9974 - val_loss: 0.4597 -